# **Model 3 Capstone Project :  RAG (Retrieval-Augmented Generation) WasteSnap**
## **Team & Project Details :** <br>
**Project Name** : WasteSnap <br>
**Project Themes** : Environment and Sustainability <br>
**Project Team Members** : <br>
Arthur Setiawan Waruwu (MC319D5Y2042) - Machine Learning Engineer - Universitas Sumatera Utara <br>		
Sakifa Indira Putri (MC319D5X2380) - Machine Learning Engineer - Universitas Sumatera Utara <br>
Diva Anggreini Harahap (MC319D5X2329) - Machine Learning Engineer - Universitas Sumatera Utara <br>		
Ahmad Wildan Naufi Raharjo (FC525D5Y0132) - Front End Back End Engineer - Politeknik Negeri Bamyuwangi <br>		
Achmad Naufal Falendra (FC525D5Y0162) - Front End Back End Engineer - Politeknik Negeri Banyuwangi <br>		
Achmad Yogi Firdani (FC525D5Y0124) - Front End Back End Engineer - Politeknik Negeri Banyuwangi <br>			

#### **Import Library**

In [1]:
import os
import logging
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import Chroma

Import semua library yang dibutuhkan untuk proses ekstraksi, chunking, embedding, dan penyimpanan data ke ChromaDB.

In [2]:
# setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

pdf_folder = 'DatasetPDF'
all_documents = []

Konfigurasi sistem logging untuk monitoring proses dan definisi variabel awal seperti folder penyimpanan PDF dan list dokumen.

#### **PDF Loader**

In [3]:
pdf_folder = 'DatasetPDF'
all_documents = []

for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        file_path = os.path.join(pdf_folder, filename)
        try:
            print(f"Memuat: {filename}")
            loader = PyMuPDFLoader(file_path)
            documents = loader.load()
            all_documents.extend(documents)
        except Exception as e:
            print(f"Gagal load {filename}: {e}")


logging.getLogger("pypdf").setLevel(logging.ERROR)
print(f"Total halaman yang berhasil dimuat: {len(all_documents)}")
print(f"Total file PDF yang berhasil diproses: {len(set([doc.metadata['source'] for doc in all_documents]))}")


Memuat: ampas-kopi-waste.pdf
Memuat: ANALISA_JENIS_LIMBAH_KAYU_PADA_INDUSTRI_PENGOLAHAN.pdf
Memuat: Analisis Limbah Sapi yang Berpotensi Mencemari Lingkungan dan Menularkan Penyakit pada Masyarakat.pdf
Memuat: buah-waste.pdf
Memuat: cangkang-telur.pdf
Memuat: Dampak Karet.pdf
Memuat: DAMPAK_NEGATIF_AKIBAT_DARI_LIMBAH_PABRIK.pdf
Memuat: Dampak_Negatif_Kardus_Lingkungan_Lengkap.pdf
Memuat: Daur Ulang sampah plastik.pdf
Memuat: e-waste-2.pdf
Memuat: e-waste-3.pdf
Memuat: e-waste-4.pdf
Memuat: e-waste-5.pdf
Memuat: e-waste-sampah.pdf
Memuat: e-waste.pdf
Memuat: Edukasi Pemanfaatan Ampas Kopi .pdf
Memuat: Elemen Interior Berbahan Baku Pengolahan Sampah Styrofoam dan Sampah Kulit Jeruk.pdf
Memuat: ewasete.pdf
Memuat: IDENTIFIKASI KARAKTERISTIK SAMPAH ELEKTRONIK.pdf
Memuat: IMPLEMENTASI RECYCLE LIMBAH KAYU HASIL PRODUKSI PLYWOOD DAN WORK WOOD MENJADI BAHAN BAKU PRODUK PARTICLE BOARD DI PT.pdf
Memuat: kaca-waste-2.pdf
Memuat: kaca-waste.pdf
Memuat: kain-3.pdf
Memuat: kain.pdf
Memuat: kalenglog

Ini akan memuat semua file PDF dari folder `DatasetPDF`, untuk setiap file PDF, dokumen akan diekstrak menggunakan `PyMuPDFLoader` dan hasil ekstraksi tiap halaman akan digabung dalam list `all_documents`. Jika ada file yang gagal dimuat, akan muncul pesan error dan setelah prosesnya selesai, maka akan menampilkan total halaman yang berhasil dimuat dan total file PDF yang berhasil diproses.

#### **Text Chunking**

In [4]:
# chuncking per Paragraf
paragraph_splitter = CharacterTextSplitter(
    separator="\n\n",         
    chunk_size=500,          
    chunk_overlap=100, #ga berguna keknya, bjirlah dh pusing cok          
    keep_separator=False      
)

split_documents = paragraph_splitter.split_documents(all_documents)
print(f"Total chunks paragraf: {len(split_documents)}")

# Tampilkan 5 chunk pertama untuk verifikasi
for i, doc in enumerate(split_documents[:5]):
    print(f"\n--- Chunk {i+1} ---")
    print(f"Source file: {doc.metadata['source']}")
    print(doc.page_content)


Total chunks paragraf: 540

--- Chunk 1 ---
Source file: DatasetPDF\ampas-kopi-waste.pdf
9 
Pemanfaatan Limbah Ampas Kopi menjadi Pupuk Organik  
  
Abstrak  
  
Coffee shop merupakan salah satu tempat populer yang sering didatangi oleh berbagai 
kalangan baik anak muda, dewasa bahkan yang sudah berumur dan menjadi pilihan yang 
nyaman untuk sekedar hangout, mengerjakan tugas atau bahkan mengadakan pertemuan 
klien. Dengan semakin populernya Coffee shop mereka semakin memperbanyak cabang 
dan memiliki banyak tempat dengan nama yang baru. Akibatnya maka akan semakin 
banyak limbah Coffee seperti limbah plastik dan limbah ampas kopi. Oleh karna itu, perlu 
diadakan pengembangan dan pemanfaatan daur ulang sehingga tidak menyebabkan limbah. 
Berbeda dengan kebanyakan Coffee shop, less waste more Coffee mereka memiliki daya 
tarik dengan tidak menggunkan plastik atau 0% plastic dan mendaur ulang ampas kopi yang 
sudah terpakai menjadi pupuk tanaman. Ini merupakan inovasi yang bagus sehingga

Melakukan chuncking teks per paragraf dengan pemisah dua newline (\n\n) dan ukuran maksimal chunk 500 karakter menggunakan overlap 100 karakter bertujuan untuk memastikan setiap potongan teks memiliki konteks yang saling berhubungan.
<br>

Overlap memungkinkan sebagian teks dari akhir chunk sebelumnya diulang di awal chunk berikutnya sehingga informasi yang penting tidak terpotong atau hilang saat pemrosesan.

#### **Inisialisasi embedding model**

In [5]:
try:
    logger.info("Inisialisasi model embedding HuggingFace...")
    embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)
    logger.info("Model embedding berhasil diinisialisasi.")
except Exception as e:
    logger.error(f"Error saat inisialisasi embedding: {e}")
    raise

2025-06-02 22:45:15,977 [INFO] Inisialisasi model embedding HuggingFace...
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_26136\821173970.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)
d:\DBS\github\Capstone-Project-WasteSnap\RAG-WasteSnap\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-02 22:45:41,067 [INFO] Use pytorch device_name: cpu
2025-06-02 22:45:41,069 [INFO] Load pretrained SentenceTransform

Model embedding sudah berhasil diinisialisasi dan siap dipakai untuk proses embedding dokumen.

In [6]:
# embedding chunks
try:
    logger.info("Memulai proses embedding dokumen...")
    vectors = embedding.embed_documents([doc.page_content for doc in split_documents])
    logger.info(f"Embedding selesai. Total vectors: {len(vectors)}")
except Exception as e:
    logger.error(f"Error saat embedding: {e}")
    raise

2025-06-02 22:45:58,858 [INFO] Memulai proses embedding dokumen...
2025-06-02 22:46:30,163 [INFO] Embedding selesai. Total vectors: 540


- Bagian ini melakukan embedding untuk semua chunk dokumen `split_documents` yaitu mengubah teks menjadi representasi vektor numerik yang bisa dipakai untuk pencarian dan pemodelan lebih lanjut.
- Proses embedding dilakukan secara batch menggunakan `embedding.embed_documents()` dengan dfatar teks dari tiap chunk `doc.page_content`.
- Totalnya ada 540 vektor yang berhasil dibuat, sesuai dengan jumlah chunk dokumen yang kita punya.


In [7]:
import shutil
if os.path.exists("chroma_store"):
    shutil.rmtree("chroma_store")

# Setelah itu baru buat dan simpan vectorstore lagi
vectordb = Chroma.from_documents(
    documents=split_documents,
    embedding=embedding,
    persist_directory="chroma_store"
)

vectordb.persist()
print("Vektor embeddings chunks berhasil disimpan ke ChromaDB.")

2025-06-02 22:46:35,607 [INFO] Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Vektor embeddings chunks berhasil disimpan ke ChromaDB.


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_26136\1404940224.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


- Demi menghindari duplikasi data, kita akan menghapus folder `chroma_store` terlebih dahulu jika sudah ada, sehingga proses ini memastikan penyimpanan baru berjalan bersih.
- Kemudian, membuat kembali `vectordb` dari dokumen dan embedding yang sudah ada.
- Memanggil persist() untuk menyimpan data ke storage lokal ChromaDB.

Selanjutnya, untuk memastikan apakah data tersebut memang sudah tersimpan dan retrieval berjalan dengan baik, kita akan melakukan pencarian sederhana

In [8]:
# cek udah tersimpan atau belum
persist_directory = "chroma_store"
embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

# inisialisasi embedding sama seperti saat simpan
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# load kembali vectorstore
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

2025-06-02 22:47:22,281 [INFO] Use pytorch device_name: cpu
2025-06-02 22:47:22,283 [INFO] Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_26136\44905787.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [9]:
print(f"Jumlah vektor yang tersimpan: {vectordb._collection.count()}")

Jumlah vektor yang tersimpan: 540


2025-06-02 22:47:55,169 [INFO] Backing off send_request(...) for 0.9s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Read timed out. (read timeout=15))


Nah, sekarang kita akan melakukan pencarian sederhana.

In [29]:
# pencarian sederhana
query = "pemanfaatan sisa kopi"
results = vectordb.similarity_search(query, k=1)  

for i, doc in enumerate(results):
    print(f"\nResult {i+1}:")
    print(f"Source: {doc.metadata.get('source')}")
    print(f"Content snippet: {doc.page_content[:300]}...\n")


Result 1:
Source: DatasetPDF\Edukasi Pemanfaatan Ampas Kopi .pdf
Content snippet: 1. Menyuburkan tanah atau tanaman Kopi yang diminum biasanya menyisakan ampas yang hanya 
dibuang begitu saja setelah digunakan. Ampas kopi mempunyai banyak manfaat terutama bagi 
tumbuhan, yaitu dapat menambah asupan Nitrogen, Fosfor dan Kalium (NPK) yang dibutuhkan oleh 
tanaman sehingga dapat men...



- `similarity_search` akan mencari chunk dengan embedding yang paling dekat (mirip) dengan query.
- Parameter k=1 artinya akan mengambil 1 hasil terbaik.
- Nah, dari hasil tersebut, kita bisa melihat apakah data yang disimpan memang sudah benar dan pencarian sudah relevan.